## DIA 044: Implementación de Trazabilidad Distribuida con OpenTelemetry

En el Día 45 se añade trazabilidad distribuida a la API utilizando OpenTelemetry. Esta integración permite instrumentar la aplicación para capturar y exportar trazas de las solicitudes, lo que facilita la monitorización y el diagnóstico del rendimiento en un entorno distribuido. En este ejemplo, se configura la instrumentación de Flask con OpenTelemetry, se utiliza el exportador de Jaeger para enviar las trazas a un agente Jaeger, y se añade un exportador de consola para visualizar las trazas en tiempo real durante el desarrollo.

Código Completo (api.py)
python
Copiar
from flask import Flask, jsonify, request
import os
import logging

# Importación de OpenTelemetry
from opentelemetry import trace
from opentelemetry.sdk.resources import SERVICE_NAME, Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor, SimpleSpanProcessor, ConsoleSpanExporter
from opentelemetry.exporter.jaeger.thrift import JaegerExporter
from opentelemetry.instrumentation.flask import FlaskInstrumentor

# Configuración básica de Flask
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuración de OpenTelemetry para trazabilidad distribuida
resource = Resource.create({SERVICE_NAME: "mnist-api"})
trace.set_tracer_provider(TracerProvider(resource=resource))
tracer = trace.get_tracer(__name__)

# Configurar el exportador de Jaeger
jaeger_exporter = JaegerExporter(
    agent_host_name=os.getenv("JAEGER_AGENT_HOST", "localhost"),
    agent_port=int(os.getenv("JAEGER_AGENT_PORT", 6831)),
)
batch_processor = BatchSpanProcessor(jaeger_exporter)
trace.get_tracer_provider().add_span_processor(batch_processor)

# Configurar el exportador de consola (útil para desarrollo)
console_exporter = ConsoleSpanExporter()
simple_processor = SimpleSpanProcessor(console_exporter)
trace.get_tracer_provider().add_span_processor(simple_processor)

# Instrumentar la aplicación Flask
FlaskInstrumentor().instrument_app(app)

# ---------------------------
# Endpoints
# ---------------------------

@app.route('/health', methods=['GET'])
def health():
    """
    Health Check
    ---
    tags:
      - Health
    responses:
      200:
        description: La aplicación está funcionando correctamente.
    """
    with tracer.start_as_current_span("health-check"):
        return jsonify({"status": "ok"}), 200

@app.route('/predict', methods=['POST'])
def predict():
    """
    Predict Endpoint
    ---
    tags:
      - Prediction
    consumes:
      - multipart/form-data
    parameters:
      - in: formData
        name: file
        type: file
        required: true
        description: Imagen para predecir el dígito.
    responses:
      200:
        description: Predicción realizada.
        schema:
          type: object
          properties:
            prediction:
              type: integer
            probability:
              type: number
    """
    with tracer.start_as_current_span("prediction"):
        # Aquí se simula una predicción; en un caso real, se procesaría la imagen.
        return jsonify({"prediction": 5, "probability": 0.90}), 200

@app.route('/login', methods=['POST'])
def login():
    """
    User Login
    ---
    tags:
      - Auth
    parameters:
      - in: body
        name: credentials
        description: Credenciales de usuario para login.
        schema:
          type: object
          required:
            - username
            - password
          properties:
            username:
              type: string
            password:
              type: string
    responses:
      200:
        description: Token JWT generado.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # Para este ejemplo, se asume que las credenciales son correctas.
    from flask_jwt_extended import create_access_token
    token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify({"access_token": token}), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
Explicación del Código
Configuración de Flask y Logging:

Se configura la aplicación Flask y se inicializa el logging para capturar eventos y errores.
Instrumentación con OpenTelemetry:

Se establece un TracerProvider con un recurso que identifica el servicio como "mnist-api".
Se configuran dos exportadores:
Jaeger Exporter: Envía las trazas a un agente Jaeger (configurable mediante variables de entorno).
Console Exporter: Imprime las trazas en la consola, útil durante el desarrollo.
Se agrega la instrumentación de Flask mediante FlaskInstrumentor().instrument_app(app).
Endpoints Instrumentados:

/health:
Se crea un span llamado "health-check" que envuelve la lógica del endpoint.
/predict:
Se crea un span llamado "prediction" para capturar la ejecución de la función, simulando la predicción.
/login:
Se define un endpoint de login básico que genera un token JWT (sin instrumentación explícita, pero la instrumentación global de Flask capturará el span).
Ejecución de la Aplicación:

La aplicación se ejecuta en modo debug. En producción, se debería usar un servidor adecuado y configurar el nivel de logging y exportadores según sea necesario.